# Project 7 - Logistic Regression and Support Vector Machines

## Notebook imports and settings

In [1]:
# Python imports
import glob
from tqdm.notebook import tnrange, tqdm_notebook

# Numerical imports
import numpy as np
import pandas as pd

# Tensorflow

# Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting settings
sns.set()
sns.set_palette("colorblind")
sns.set_style("ticks")

## Load and format data

In [5]:
def convert_to_csv(folder):
    with open(f"./data/{folder}.csv", "w+") as fout:
        print("class,del1,del2,x1,y1,x2,y2,size,diag", file=fout)
        
        for file_name in tqdm_notebook(glob.glob(f"./data/{folder}/*"), desc=f"{folder}"):
            for line in open(file_name):
                line = line.split()
                line = ",".join(line)
                print(line, file=fout)
                
    df = pd.read_csv(f"./data/{folder}.csv")
    df = df.drop(["del1", "del2"], axis=1)
    df.to_csv(f"./data/{folder}.csv", index=False)

convert_to_csv("train")
convert_to_csv("test")

train = pd.read_csv("./data/train.csv")
train.head(5)

,class,x1,y1,x2,y2,size,diag
0,1,0,0,0,20,20.00,46.1
1,1,19,0,19,8,8.00,46.1
2,1,0,20,19,8,22.47,46.1
3,1,0,20,8,42,23.41,46.1
4,1,19,8,8,42,35.74,46.1
